In [7]:
import pandas as pd

In [29]:
df = pd.read_csv('articles.csv', encoding='iso-8859-1')

In [30]:
df = df.drop(columns=['Id', 'Article.Banner.Image'])
df = df.dropna()

In [31]:
df.head()

,Heading,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,Bells Nexus Air Taxi Concept Rings Changes Fo...,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive
2,Bell Helicopter Show Air Taxi Nexus,TMTPost,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive
3,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,Fredzone,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive
4,Les premiers retours dOlivier Ezratty,FrenchWeb,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive


In [47]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer

In [25]:
vectorizer = SentenceTransformer('paraphrase-MiniLM-L6-v2')

C:\Users\Rinaldo\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rinaldo\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Rinaldo\AppData\Roaming\Python\Python312\site-packages\transfo

In [40]:
import re
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

def lower_column(text):
    return text.lower()

In [43]:
df['Full_Article']= df['Full_Article'].apply(clean_text)
df['Article.Description']= df['Article.Description'].apply(clean_text)
df['Outlets']= df['Outlets'].apply(clean_text)
df['Heading']= df['Heading'].apply(clean_text)
df['Article_Type'] = df['Article_Type'].apply(clean_text)
df['Tonality'] = df['Tonality'].apply(clean_text)

In [44]:
df.head()

,Heading,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,a puzzling maneuver then freefall ntsb report ...,essex caller,the helicopter that crashed in southeast alask...,the helicopter that crashed in southeast alask...,commercial,negative
1,bells nexus air taxi concept rings changes for...,aviation week network,a year after teasing the fledgling electric ve...,a year after teasing the fledgling electric ve...,commercial,positive
2,bell helicopter show air taxi nexus,tmtpost,bell released the fullsize design of the verti...,bell released the fullsize design of the verti...,commercial,positive
3,bell dvoile la conception intgrale de son taxi...,fredzone,bell est une socieacuteteacute ameacutericaine...,bell est une socieacuteteacute ameacutericaine...,commercial,positive
4,les premiers retours dolivier ezratty,frenchweb,it was still anecdotal to observe the explosio...,it was still anecdotal to observe the explosio...,commercial,positive


In [49]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# X = vectorizer.encode(df['Full_Article'].tolist())

C:\Users\Rinaldo\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rinaldo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Rinaldo\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base

In [79]:
X = tokenizer.encode(df['Heading'].tolist(), return_tensors="pt")

In [80]:
columns = ['Full_Article', 'Article.Description', 'Outlets', 'Heading', 'Tonality']

In [81]:
import numpy as np
new_df = pd.DataFrame()
X = []

In [82]:
for col in columns:
    X.append(tokenizer.encode(df[col].tolist()))

In [83]:
X = np.array(X)

In [84]:
X.shape

(5, 3875)

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score